In [13]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

In [2]:
df = pd.read_json('prepped_data.json').sort_index().reset_index()
print(df.shape)
display(df.head(2))
df.tail(2)

(4842, 10)


,index,age,num_comments,score,text,timestamp,weekday_posted,hour_posted,log_score,log_comments
0,0,300,1588,21720,China Killing Prisoners To Harvest Organs For ...,2019-06-19 11:49:08,2,11,9.985989,7.370237
1,1,240,402,2661,Muslim family dragged out of Belgian embassy i...,2019-06-19 12:05:31,2,12,7.886457,5.996477


,index,age,num_comments,score,text,timestamp,weekday_posted,hour_posted,log_score,log_comments
4840,5098,960,126,2007,Oil companies have played a major role in clim...,2018-05-16 01:17:21,2,1,7.604396,4.836361
4841,5099,780,492,1060,Nikki Haley denies Gaza violence is related to...,2018-05-16 04:31:37,2,4,6.966024,6.198499


In [3]:
import math
tot = 4842
split_index = math.ceil(0.2 * tot,)
split_index

969

In [4]:
test = df[df.index < split_index]
train = df[df.index >= split_index]

In [7]:
tfidf = TfidfVectorizer(max_features=2000)
tfidf.fit(train.text)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=2000, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [8]:
train_tfidf_vecs = pd.DataFrame(tfidf.transform(train.text).toarray())
train_tfidf_df = train[['age', 'weekday_posted', 'hour_posted', 'score', 'num_comments']].reset_index(drop=True)
display(train_tfidf_df.head())
train_tfidf_df = pd.concat([train_tfidf_df, train_tfidf_vecs], axis=1)
train_tfidf_df.head()

,age,weekday_posted,hour_posted,score,num_comments
0,360,0,11,1582,128
1,540,0,8,2323,126
2,420,0,10,792,68
3,60,0,16,268,58
4,1140,6,22,7472,423


,age,weekday_posted,hour_posted,score,num_comments,0,1,2,3,4,...,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999
0,360,0,11,1582,128,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,540,0,8,2323,126,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,420,0,10,792,68,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,60,0,16,268,58,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1140,6,22,7472,423,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
test_tfidf_vecs = pd.DataFrame(tfidf.transform(test.text).toarray())
test_tfidf_df = test[['age', 'weekday_posted', 'hour_posted', 'score', 'num_comments']].reset_index(drop=True)
display(test_tfidf_df.head())
test_tfidf_df = pd.concat([test_tfidf_df, test_tfidf_vecs], axis=1)
test_tfidf_df.head()

,age,weekday_posted,hour_posted,score,num_comments
0,300,2,11,21720,1588
1,240,2,12,2661,402
2,660,2,5,46977,3320
3,360,2,10,1474,202
4,660,2,5,2665,1336


,age,weekday_posted,hour_posted,score,num_comments,0,1,2,3,4,...,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999
0,300,2,11,21720,1588,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,240,2,12,2661,402,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,660,2,5,46977,3320,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,360,2,10,1474,202,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,660,2,5,2665,1336,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
train_tfidf_df.to_json('tfidf_train.json')
test_tfidf_df.to_json('tfidf_test.json')

In [12]:
# pd.read_json(open("tfidf_train.json", "r", encoding="utf8"))   # Test to confirm correct json storage

In [16]:
numerical_features = train[['age']].columns
categorical_features = train[['weekday_posted',  'hour_posted']]